In [15]:
from PIL import Image, ImageDraw, ImageFont
import textwrap
Image.MAX_IMAGE_PIXELS = None

from tqdm.auto import tqdm
import numpy as np
import pandas as pd
import requests
import shutil

import os.path
import io
import pickle
from urllib.parse import urlparse
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.http import MediaIoBaseDownload

In [2]:
SCOPES = ['https://www.googleapis.com/auth/drive']
creds = None
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)
drive = build('drive', 'v3', credentials=creds)

In [3]:
data = pd.read_csv('BECMPNA.csv')
data.head()

,Timestamp,Full Name.,Roll No.,Picture,Quote,Groupfie
0,6/20/2020 22:56:27,Shalom Marshal Dbritto,7,https://drive.google.com/open?id=1h-cJ_6kPMOKE...,No matter what..\ndo{\nlive_life(❤);\n} \nwhil...,https://drive.google.com/open?id=1SXptCEvLaFZO...
1,6/28/2020 1:03:13,Vinit Anishkumar Masrani,64,https://drive.google.com/open?id=1DcIy7NkwXy3V...,If path be beautiful let us not ask where it l...,NaN
2,7/5/2020 17:22:55,Harsh Jaykumar Jalan,58,https://drive.google.com/open?id=1PndDuf6247wD...,Life Goals = $$$ and 5318008,https://drive.google.com/open?id=1JsJIll2Jwdul...
3,7/7/2020 23:14:34,Riya Walter Dabre,5,https://drive.google.com/open?id=15dyxfneRpXLW...,When I tell everyone that I am an engineer 😅🤟.,https://drive.google.com/open?id=11gyVFa9rxnz4...
4,6/30/2020 20:42:20,Lenwyn,46,https://drive.google.com/open?id=1klq5b59FjqPU...,Good coffee for good communication.,https://drive.google.com/open?id=1wiUu46dMceQt...


In [4]:
def getId(url):
    urls = url.split(',')
    res = []
    for link in urls:
        res.append(urlparse(link).query[3:])
    return res

In [5]:
data['ImageId'] = data['Picture '].apply(lambda x : getId(x))

In [21]:
def generateStudent(target,name,img,quote,index):
    
    # get image
#     img = img.replace('open','uc')
#     req = requests.get(img,stream=True)
#     print(name,req.status_code)
#     with open('student.png', 'wb') as out_file:
#         shutil.copyfileobj(req.raw, out_file)
    
    # google api to download file into student.png
    request = drive.files().get_media(fileId=img)
    fh = io.FileIO("student.png", 'wb')
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print(name)    
        
    filename = name+str(index)
        
    student = Image.open('student.png')
    if student.size[0]>student.size[1]:
        # save vertical image with 90 shifts
        getImg(target,name,student.rotate(270,expand=True),quote).save('testFolder/'+filename+'rotate270.png')
        getImg(target,name,student.rotate(90,expand=True),quote).save('testFolder/'+filename+'rotate90.png')
    getImg(target,name,student,quote).save('testFolder/'+filename+'.png')
        

def getImg(target,name,student,quote):
    
    # Clear white workspace
    d = ImageDraw.Draw(target)
    d.rectangle(((190,75 ), (900,1000)), fill="white")
    
    # Paste student image in target
    student.thumbnail( (700,700), Image.ANTIALIAS)
    target.paste(student,((1080-student.size[0])//2,130))
    
    # Name Rectangle
    d.rectangle(((190,800 ), (890,900)), fill="#fff2e6")
    
    # Add Name
    namefont = ImageFont.truetype("Abys-Regular.otf", 45)
    w,h = d.textsize(name,namefont)
    d.text(((1080-w)/2,820), name, fill=(0,0,0),font=namefont)
    
    # Add Quote
    quotefont = ImageFont.truetype("Symbola.ttf", 30)
    currentHt,pad = 930,30
    para = textwrap.wrap(quote, width=60)
    for line in para:
        w,h = d.textsize(line,quotefont)
        d.text(((1080-w)/2,currentHt), line, fill=(0,0,0),font=quotefont)
        currentHt += pad
    
    return target

In [22]:
# CMPN : targetPink
# EXTC : targetBlue
# IT : targetGreen

for i in tqdm(range(data.shape[0])):
    target = Image.open('targetPink.jfif')
    name = data.iloc[i]['Full Name.']
    img = data.iloc[i]['ImageId']
    quote = data.iloc[i]['Quote']
    
    for j in range(len(img)):
        generateStudent(target,name,img[j],quote,j)

Shalom Marshal Dbritto
Vinit Anishkumar Masrani
Harsh Jaykumar Jalan
Harsh Jaykumar Jalan
Riya Walter Dabre
Riya Walter Dabre
Lenwyn
Nash Morris Dsilva
Merryl Tuscano
Pankaj Masaye
Sakshi Tholar
Jowin Jestine
Leo Amit Dcosta 
Leo Amit Dcosta 
Leo Amit Dcosta 
Gwyn Mendonca
Sneha Jha
Sneha Jha
Glen Johnson Pinto
Joyan Serrao
Joyan Serrao
Sanil Fernandes
Sanil Fernandes
Jaison Menezes
Priya Nawal
Anushka Kamath
Jaison Menezes
Shervin David Fernandez
Shervin David Fernandez
Maroof Ibrahim Khatib
Jay Jokim D'mello
Diparth Shah
Diparth Shah
Susan Sara Thomas
Jay Jokim D'mello
Jay Jokim D'mello
Zenas Mukadom
Zenas Mukadom
Steve Rodrigues
Nirajkumar Yadav
Nirajkumar Yadav
Warren Fernandes
Rion Tuscano
Suraj Negi
Suraj Negi
Velina Albert Rodrigues
Clive Pereira 
Clive Pereira 
Charles Mathew
Joyson Almeida
Gautam Virendra Maurya
Gautam Virendra Maurya
Shrey Singh
Shrey Singh
Shakhaf J. Joseph
Preethi Manuvel Bhutelo
Preethi Manuvel Bhutelo
Preethi Manuvel Bhutelo
Archana Yadav
Archana Yadav
Ly